In [2]:
# Import bibliotek
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep
import ipywidgets as widgets


In [3]:
# funkcja wykorzystana w kodach:

def zmiana_na_float (cecha):
    """ 
    Funkcja do zmiany danych poszczegolnych cech na float z odcięciem jednostki.
      
    Parametry:
    ----------
    cecha (str): Liczba podana jako ciąg znaków 
  
    Wynik:
    ------
    float: Ciąg znaków zamieniony na float.
    
    Przykład:
    ---------
    zmiana_na_float('50,2 m2')->50.2
      
    """
    
    if isinstance (cecha, str): # jeśli cecha jest stringiem
        return float(cecha.rsplit(' ',1)[0].replace(' ','').replace(',','.'))     #rsplit-rozdzielam po spacji, ale tylko jednej spacji licząc od końca. Jeśli w stringu będzie kilka spacji, to podzieli tylko po tej ostatniej 
    else:
        return cecha

## 1. Kod do pobrania szczegółów ogłoszeń (widocznych z poziomu strony głównej)-MIESZKANIA

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam jedynie dane z pierwszej strony zwróconej przez Otodom. Są to dane widoczne z poziomu tej strony ( bez zagłębiania się w szczegóły, które można zobaczyć tylko po wejściu w konkretne oferty)
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom, jak również szczegółowe dane widoczne tylko po wejściu w konkretną ofertę 


In [4]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strony)
adres_strony=input('Podaj adres strony www: ')
zawartosc = requests.get(adres_strony)

Podaj adres strony www: https://www.otodom.pl/sprzedaz/mieszkanie/swarzedz/?search%5Bdist%5D=0&search%5Bsubregion_id%5D=458&search%5Bcity_id%5D=81


In [5]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [6]:
f = csv.writer(open('oferty_mieszkania.csv', 'w'))

In [ ]:
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


for item in wszystkie_oferty_items: # dla każdej cechy, którą chcę wydobyć, szukam jej klasy na stronie Otodom
    naglowek=item.find(class_='text-nowrap hidden-xs').get_text()
    liczba_pokoi=item.find(class_="offer-item-rooms hidden-xs").get_text()
    powierzchnia=item.find(class_="hidden-xs offer-item-area").get_text()
    cena=item.find(class_="offer-item-price").get_text()
    cena_zlm2=item.find(class_="hidden-xs offer-item-price-per-m").get_text()
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        
    # poniższe cechy zamieniam na float wg.wczesniej napisanej funkcji
    liczba_pokoi=zmiana_na_float(liczba_pokoi)
    powierzchnia=zmiana_na_float(powierzchnia)
    cena=zmiana_na_float(cena.strip())
    cena_zlm2 =zmiana_na_float(cena_zlm2)

    print('tytuł: ', naglowek)
    print('liczba_pokoi: ', liczba_pokoi)
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zl_m2): ', cena_zlm2)
    print(adres_www, '\n')
    
    f.writerow([naglowek, liczba_pokoi,powierzchnia, cena, cena_zlm2, adres_www ]) ## zapisanie do csv

In [ ]:
## zapisanie do CSV
f.writerow(['tytuł', 'liczba_pokoi', 'powierzchnia','cena', 'cena_zlm2', 'adres_www' ])

## 2. Kod do pobrania szczegółów zawartych wewnątrz konkretnych ogłoszeń  - MIESZKANIA

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, powierzchni, rodzaju nieruchomości - w tym wypadku mieszkania, itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam dane z pierwszej strony zwróconej przez Otodom, przy czym są to dane szczegółowe widoczne po otwarciu konkretnych ofert
- po pobraniu danych, zapisuję je do pliku w formacie CSV
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom

In [9]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strona)
adres_strony=input('Podaj adres strony www: ')
zawartosc = requests.get(adres_strony)

Podaj adres strony www: https://www.otodom.pl/sprzedaz/mieszkanie/swarzedz/?search%5Bdist%5D=0&search%5Bsubregion_id%5D=458&search%5Bcity_id%5D=81


In [10]:
zawartosc.status_code

200

In [11]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
# kod do zaczytania szczegółów zawartych w ofertach (przy przeglądaniu strony www widoczne są tylko po wejściu w daną ofertę)
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

for item in wszystkie_oferty_items:
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        zawartosc2=requests.get(adres_www)
        soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
        szczegoly=soup2.find_all(class_='article-offer')
        for item in szczegoly:       
            adres=item.find(class_='address-lane').get_text()
            powierzchnia=item.find(class_='big').get_text()
            cena=item.find(class_="box-price-value").get_text()
            try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                pietro=item.find(class_='param_floor_no').get_text()
            except AttributeError:
                pietro='brak informacji'
            liczba_pokoi=item.find(class_='room-lane').get_text()
            opis=item.find(itemprop='description').get_text()


        powierzchnia=zmiana_na_float(powierzchnia)   
        cena=zmiana_na_float(cena.strip())
        cena_zl_m2=round(cena/powierzchnia,2)
        liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]



    print('adres: ', adres.strip())
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zł/m2): ', cena_zl_m2)
    print('pietro: ', pietro)
    print('liczba_pokoi: ', liczba_pokoi[0])
    print('opis: ', opis)
    print(adres_www, '\n')

    f.writerow([adres, powierzchnia,cena, cena_zl_m2, pietro, liczba_pokoi[0], opis ]) ## zapisanie do csv
        

##  3. Kod do pobrania ofert z kilku stron - MIESZKANIA

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w poprzednich kodach czytałam jedynie dane widoczne na pierwszej stronie zwrócoej przez Otodom, teraz zaczytam dane z wielu stron. Zrobię to na 2 sposoby:
    - w pierwszym kodzie użytkownik sam będzie podawał, z ilu stron chce uzyskać dane ( np. Otodom zwróci 10 stron, a użytkownik uzna, że chce dane tylko z 2 stron). Poza tym użytkownik będzie mógł wybrać, z których stron chce otrzymać dane, wpisując linki do tych stron
    - w drugim kodzie automatycznie zaczytam dane z wszystkich stron zwróconych przez Otodom
- dodatkowo uporządkuję pobrane dane wyorzystując bibliotekę Pandas (do csv będą eksportowane uporządkowane dane)


### 3a. Użytkownik podaje, z których stron chce pobrać dane 

In [ ]:
strony=[]
for strona in range(0,int(input("Podaj, ile stron chcesz przejrzeć: "))):
    strona=input(str("Wpisz pełen adres strony- cały link: "))
    strony.append(strona)
    print('strony: ', strony)

In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
dodatkowe=[]
opisy=[]

    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                    pietro=item.find(class_='param_floor_no').get_text()
                except AttributeError:
                    pietro='brak informacji'
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                dodatkowe_parametry=soup2.find_all(class_='sub-list')
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':', maxsplit=1)#maxsplit=1 dzieli tylko po pierwszym dwukropku
                        slownik[klucz]=wartosc
                

                    powierzchnia=zmiana_na_float(powierzchnia)   
                    cena=zmiana_na_float(cena.strip())
                    cena_zl_m2=round(cena/powierzchnia,2)
                    liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]
                    

                    print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                    print('powierzchnia (m2): ', powierzchnia)
                    print('cena (zł): ', cena)
                    print('cena (zł/m2): ', cena_zl_m2)
                    print('pietro: ', pietro)
                    print('liczba_pokoi: ', liczba_pokoi[0])
                    for klucz,wartosc in slownik.items():
                        print(klucz,': ',wartosc)
                    print('opis: ', opis)
                    print(adres_www, '\n')

                    # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandas
                    adresy.append(adres.strip())
                    powierzchnie.append(powierzchnia)
                    ceny.append(cena)
                    ceny_zlm2.append(cena_zl_m2)
                    pietra.append(pietro)
                    pokoje.append(liczba_pokoi[0])
                    opisy.append(opis)
                    dodatkowe.append(slownik)


### Obróbka  danych - Pandas - MIESZKANIA

In [ ]:
oferty=pd.DataFrame()
for x in range(0,len(adresy)):
    sr=pd.Series([adresy[x],powierzchnie[x],ceny[x],ceny_zlm2[x], pietra[x],pokoje[x], opisy[x] ],\
                 index=['adres','powierzchnia (m2)','cena (zł)','Cena (zł/m2)', 'pietro', 'liczba_pokoi', 'opis'])
    sr_dodatkowe=pd.Series(dodatkowe[x])
    sr=pd.concat([sr,sr_dodatkowe])
    oferty=pd.concat([oferty,sr],axis=1,ignore_index=True)

In [71]:
oferty.index

Index(['Cena (zł/m2)', 'Forma własności', 'Okna', 'Rodzaj zabudowy',
       'Rok budowy', 'Rynek', 'adres', 'cena (zł)', 'liczba_pokoi', 'opis',
       'pietro', 'powierzchnia (m2)'],
      dtype='object')

In [72]:
oferty=oferty.T

In [74]:
kolejnosc_kolumn=['adres','Rynek','powierzchnia (m2)','cena (zł)','Cena (zł/m2)','Forma własności',\
'liczba_pokoi', 'pietro', 'Rodzaj zabudowy', 'Stan wykończenia', 'Czynsz', 'opis']

In [75]:
# ustawiam kolumny w wybranej kolejności. 
#Jeśli nieruchomość nie posiada danych w zakresie określonej cechy, w odpowiedniej komórce pojawi się 'Nan'
mieszkania_oferty=oferty.reindex(columns=kolejnosc_kolumn)

In [ ]:
mieszkania_oferty

In [ ]:
# usuwam duplikaty
mieszkania_oferty.drop_duplicates(subset=['powierzchnia (m2)', 'cena (zł)','pietro','liczba_pokoi' ])

In [80]:
#usuwam wyraz'Piętro' z kolumny 'pietro'
mieszkania_oferty['pietro']=mieszkania_oferty['pietro'].apply(lambda x: x.replace('Piętro',''))

In [82]:
# usuwam 'escape characters' z kolumny 'opis'
mieszkania_oferty.loc[:,'opis'] = mieszkania_oferty.opis.str.replace('\n','',regex=True)

In [ ]:
# zapisanie pobranych ofert do pliku CSV
# mieszkania.to_csv(r'/home/kinga/python/Web_scrapping/mieszkania_oferty.csv')

### 3b. Kod, który automatycznie  pobiera dane ze wszystkich stron zwróconych przez Otodom

In [17]:
strona=input('Wpisz adres strony internetowej: ')
strony=[strona]
page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager')


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=item.find(class_='current').get_text()
    print ('liczba stron: ', liczba_str)


Wpisz adres strony internetowej: https://www.otodom.pl/sprzedaz/mieszkanie/swarzedz/?search%5Bfilter_float_m%3Ato%5D=55&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bsubregion_id%5D=458&search%5Bcity_id%5D=81
liczba stron:  3


In [18]:
#i=1
for i in range(2,int(liczba_str)+1): ## zaczynam od 2 dlatego, że dodawanie numerów w adresie strony zaczyna się od 2  
    strony.append(strona +'&page=' + str(i))  # każda kolejna strona ma końcówkę '&page=(nr_str)' w stosunku do strony pierwszej, poczynając od '&page=2' 

In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
dodatkowe=[]
opisy=[]

    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                if soup2.find_all(class_='param_floor_no'):
                    pietro=item.find(class_='param_floor_no').get_text().replace('Piętro','')
                else:
                    pietro='brak informacji'   
                if soup2.find_all(class_='room-lane'):
                    liczba_pokoi=item.find(class_='room-lane').get_text()
                    liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()][0]
                if soup2.find_all(itemprop='description'):
                    opis=item.find(itemprop='description').get_text()
                else:
                    opis='brak informacji'  
                dodatkowe_parametry=soup2.find_all(class_='sub-list')
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':', maxsplit=1)#maxsplit=1 dzieli tylko po pierwszym dwukropku
                        slownik[klucz]=wartosc
                                          
                

                    powierzchnia=zmiana_na_float(powierzchnia)   
                    cena=zmiana_na_float(cena.strip())
                    cena_zl_m2=round(cena/powierzchnia,2)
                  
                    

                    print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                    print('powierzchnia (m2): ', powierzchnia)
                    print('cena (zł): ', cena)
                    print('cena (zł/m2): ', cena_zl_m2)
                    print('pietro: ', pietro)
                    print('liczba_pokoi: ', liczba_pokoi)
                    for klucz,wartosc in slownik.items():
                        print(klucz,': ',wartosc)
                    print('opis: ', opis)
                    print(adres_www, '\n')

                    # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandas
                    adresy.append(adres.strip())
                    powierzchnie.append(powierzchnia)
                    ceny.append(cena)
                    ceny_zlm2.append(cena_zl_m2)
                    pietra.append(pietro)
                    pokoje.append(liczba_pokoi)
                    opisy.append(opis)
                    dodatkowe.append(slownik)


## 4. Kod do pobrania ofert dla różnego typu nieruchomości: MIESZKANIA, DOMY, LOKALE UŻYTKOWE, DZIAŁKI 

- kod pobiera oferty w.w. typów nieruchomości ( dostępne są różne atrybuty w zależności od rodzaju nieruchomości)
- po pobraniu dane są umieszczane w tabeli tworzonej przy wykorzystaniu biblioteki Pandas. Dla każdego typu nieruchomości tworzona jest inna tabela
- po obróbce, dane zapisywane są w określonej lokalizacji w pliku o nazwie wybranej przez użytkownika

In [33]:
strona=input('Wpisz adres strony internetowej: ')
strony=[strona]
liczba_wybranych_str=int(input("Podaj, ile stron chcesz przejrzeć: "))
plik_wynikowy=input("Podaj nazwę pliku, do którego chcesz zapisać: ")

page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') 


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=int(item.find(class_='current').get_text())
    print ('Liczba wybranych stron to: ', liczba_wybranych_str)
    print ('liczba dostępnych stron to: ', liczba_str)

    # ograniczenie- użytkownik może pobrać maksymalnie 10 stron
    if liczba_wybranych_str<=liczba_str and liczba_wybranych_str<=10:
        for i in range(2,liczba_wybranych_str+1):
            strony.append(strona +'&page=' + str(i))
    else:
        for i in range(2,11):
            strony.append(strona +'&page=' + str(i))
        print('Maksymalnie możesz pobrać 10 stron- tyle zostanie pobranych!')



adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
wielkosc_dzialki=[]
rodzaj_dzialki=[]
dodatkowe=[]
opisy=[]


for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')

    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()                     
                if soup2.find_all(class_='param_floor_no'):
                    pietro=item.find(class_='param_floor_no').get_text().replace('Piętro','')
                else:
                    pietro='brak informacji'      
                if soup2.find_all(class_='room-lane'):
                    liczba_pokoi=item.find(class_='room-lane').get_text()
                    liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()][0]
                else: 
                    liczba_pokoi='brak informacji' 
                if soup2.find_all(class_='param_terrain_area'):
                    powierzchnia_dzialki=item.find(class_='param_terrain_area').get_text().replace('Powierzchnia działki ','').replace('m²','')
                else:
                    powierzchnia_dzialki='brak informacji'
                if soup2.find_all(class_='param_type'):
                    typ_dzialki=item.find(class_='param_type').get_text().replace('Typ działki', '')
                else:
                    typ_dzialki='brak informacji'
                if soup2.find_all(itemprop='description'):
                    opis=item.find(itemprop='description').get_text()
                else:
                    opis='brak informacji'   

                dodatkowe_parametry=soup2.find_all(class_='sub-list')
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':', maxsplit=1)#maxsplit=1 dzieli tylko po pierwszym dwukropku
                        slownik[klucz]=wartosc



                    powierzchnia=zmiana_na_float(powierzchnia)   
                    cena=zmiana_na_float(cena.strip())
                    cena_zl_m2=round(cena/powierzchnia,2)

                    # zapisuję dane do takiej postaci, abym mogła je dalej zapisać w Pandas
                    adresy.append(adres.strip())
                    powierzchnie.append(powierzchnia)
                    ceny.append(cena)
                    ceny_zlm2.append(cena_zl_m2)
                    pietra.append(pietro)
                    pokoje.append(liczba_pokoi)
                    wielkosc_dzialki.append(powierzchnia_dzialki)
                    rodzaj_dzialki.append(typ_dzialki)
                    opisy.append(opis)
                    dodatkowe.append(slownik)

oferty=pd.DataFrame()
for x in range(0,len(adresy)):
    sr=pd.Series([adresy[x],powierzchnie[x],ceny[x],ceny_zlm2[x], pietra[x],pokoje[x], wielkosc_dzialki[x], rodzaj_dzialki[x], opisy[x] ],\
                 index=['Adres','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)', 'Pietro', 'Liczba_pokoi','Powierzchnia działki (m2)', 'Typ działki', 'Opis'])
    sr_dodatkowe=pd.Series(dodatkowe[x])
    sr=pd.concat([sr,sr_dodatkowe])
    oferty=pd.concat([oferty,sr],axis=1,ignore_index=True, sort=False)

oferty=oferty.T

 # usuwam 'escape characters' z kolumny 'opis'
oferty.loc[:,'Opis'] = oferty.Opis.str.replace('\n','',regex=True)
 # usuwam duplikaty
oferty.drop_duplicates()


# Tworzę odpowiedni data frame w zależności od typu nieruchomości
if oferty[oferty['Adres'].str.contains('Mieszkanie')].shape[0]>0:
    kolejnosc_kolumn=['Adres','Rynek','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)','Forma własności',
                      'Liczba_pokoi', 'Pietro', 'Rodzaj zabudowy', 'Stan wykończenia', 'Czynsz', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Adres'].str.contains('Dom ')].shape[0]>0:
    kolejnosc_kolumn=['Adres','Rynek','Powierzchnia (m2)','Powierzchnia działki (m2)','Cena (zł)',
                      'Cena (zł/m2)', 'Rodzaj zabudowy','Stan wykończenia', 'Opis' ]
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Adres'].str.contains('Działka')].shape[0]>0:
    kolejnosc_kolumn=['Adres', 'Powierzchnia (m2)', 'Cena (zł)', 'Cena (zł/m2)', 'Typ działki', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Adres'].str.contains('użytkowy')].shape[0]>0:
    kolejnosc_kolumn=['Adres','Rynek','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)','Piętro','Stan wykończenia', 
                      'Przeznaczenie lokalu', 'Rok budowy', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)


sciezka='/home/kinga/python/Web_scrapping/'+plik_wynikowy+'.xlsx'
# zapisanie do pliku Excel 
oferty.to_excel(sciezka, index=False)
print ('GOTOWE :) Plik został zapisany w lokalizacji: ', sciezka)


Wpisz adres strony internetowej: https://www.otodom.pl/wynajem/lokal/radom/?search%5Bfilter_enum_use_types%5D%5B0%5D=services&search%5Bfilter_enum_use_types%5D%5B1%5D=office&search%5Bfilter_enum_use_types%5D%5B2%5D=retail&search%5Bfilter_enum_use_types%5D%5B3%5D=gastronomy&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bsubregion_id%5D=419&search%5Bcity_id%5D=196
Podaj, ile stron chcesz przejrzeć: 6
Podaj nazwę pliku, do którego chcesz zapisać: Radom_nowe1
Liczba wybranych stron to:  6
liczba dostępnych stron to:  6
GOTOWE :) Plik został zapisany w lokalizacji:  /home/kinga/python/Web_scrapping/Radom_nowe1.xlsx


## 5. Kod do pobierania ofert po zmianie strony intrnetowej

- kod działa w oparciu o zmienioną stronę Otodom
- kod pobiera oferty różnych typów nieruchomości ( dostępne są różne atrybuty w zależności od rodzaju nieruchomości)
- dodatkowo dodano pasek postępu, który pokazuje, ile stron zostało sparsowanych
- po pobraniu dane są umieszczane w tabeli tworzonej przy wykorzystaniu biblioteki Pandas. Dla każdego typu nieruchomości tworzona jest inna tabela
- po obróbce, dane zapisywane są w określonej lokalizacji w pliku o nazwie wybranej przez użytkownika

In [10]:
strona=input('Wpisz adres strony internetowej: ')
strony=[strona]
liczba_wybranych_str=int(input("Podaj, ile stron chcesz przejrzeć: "))
plik_wynikowy=input("Podaj nazwę pliku, do którego chcesz zapisać: ")

page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')


cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') 


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=int(item.find(class_='current').get_text())
    print ('Liczba wybranych stron to: ', liczba_wybranych_str)
    print ('liczba dostępnych stron to: ', liczba_str)

    # ograniczenie- użytkownik może pobrać maksymalnie 10 stron
    if liczba_wybranych_str<=liczba_str and liczba_wybranych_str<=10:
        for i in range(2,liczba_wybranych_str+1):
            strony.append(strona +'&page=' + str(i))
    else:
        for i in range(2,11):
            strony.append(strona +'&page=' + str(i))
        print('Maksymalnie możesz pobrać 10 stron- tyle zostanie pobranych!')


rodzaje_nieruchomosci=[]
adresy=[]
ceny=[]
dodatkowe=[]
opisy=[]

x=0
pasek_postepu = widgets.IntProgress(min=x, max=liczba_wybranych_str) # pasek postępu 
opis_paska_postepu= widgets.Label(value='Liczba pobranych stron',layout=widgets.Layout(width='auto') )
liczba_sparsowanych_stron =widgets.Label() 
kontener_paska_postepu= widgets.HBox([opis_paska_postepu,pasek_postepu, liczba_sparsowanych_stron])
display(kontener_paska_postepu)



for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')

    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


    for item in wszystkie_oferty_items:
        rodzaj_nieruchomosci=item.find(class_='text-nowrap hidden-xs').get_text().split(':')[0]
        cena=zmiana_na_float(item.find(class_="offer-item-price").get_text().strip()) # strip - usuwam spacje na początku i końcu
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') 
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(id='root') 
            for item in szczegoly:       
                adres=item.find (class_='css-xpo2mg-baseStyle-Address-contentStyle').get_text()      
                if soup2.find_all(class_='section-description'): 
                    opis=item.find(class_='section-description').get_text()
                else:
                    opis='brak informacji'   
                dodatkowe_parametry=soup2.find_all(class_='css-1v9yl6n-AdOverview-className') 
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':', maxsplit=1)#maxsplit=1 dzieli tylko po pierwszym dwukropku
                        slownik[klucz]=wartosc
                    
#                     print('Rodzaj nieruchomości: ', rodzaj_nieruchomosci)
#                     print('Adres: ', adres)
#                     print('Cena (zł): ', cena)
#                     for klucz,wartosc in slownik.items():
#                         print(klucz,': ',wartosc)
#                     print('Opis: ', opis)
#                     print(adres_www, '\n')

                    # zapisuję dane do takiej postaci, abym mogła je dalej zapisać w Pandas
                    rodzaje_nieruchomosci.append(rodzaj_nieruchomosci)
                    adresy.append(adres)
                    ceny.append(cena)
                    opisy.append(opis)
                    dodatkowe.append(slownik)
    x=x+1
    pasek_postepu.value=x
    liczba_sparsowanych_stron.value=str(x) # Label musi być stringiem

oferty=pd.DataFrame()
for x in range(0,len(adresy)):
    sr=pd.Series([rodzaje_nieruchomosci[x],adresy[x],ceny[x], opisy[x] ],\
                 index=['Rodzaj nieruchomosci','Adres','Cena (zł)', 'Opis'])
    sr_dodatkowe=pd.Series(dodatkowe[x])
    sr=pd.concat([sr,sr_dodatkowe])
    oferty=pd.concat([oferty,sr],axis=1,ignore_index=True, sort=False)

oferty=oferty.T

 # usuwam 'escape characters' z kolumny 'opis'
oferty.loc[:,'Opis'] = oferty.Opis.str.replace('\n','',regex=True)
 # usuwam duplikaty
oferty.drop_duplicates()
oferty.rename(columns={'Powierzchnia': 'Powierzchnia (m2)'}, inplace=True)
oferty['Powierzchnia (m2)']=oferty['Powierzchnia (m2)'].apply(lambda x: zmiana_na_float(x)) # zmieniam powierzchnię na float i usuwam 'm2'
oferty['Cena (zł/m2)']=oferty.apply(lambda x: round(x['Cena (zł)']/x['Powierzchnia (m2)'],2),axis=1)

# Tworzę odpowiedni data frame w zależności od typu nieruchomości
if oferty[oferty['Rodzaj nieruchomosci'].str.contains('Mieszkanie')].shape[0]>0:
    kolejnosc_kolumn=['Rodzaj nieruchomosci','Adres','Rynek','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)',
                      'Forma własności','Liczba_pokoi', 'Piętro', 'Rodzaj zabudowy', 'Stan wykończenia', 
                      'Czynsz', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('Dom ')].shape[0]>0:
    oferty.rename(columns={'Powierzchnia działki': 'Powierzchnia działki (m2)'}, inplace=True)
    oferty['Powierzchnia działki (m2)']=oferty['Powierzchnia działki (m2)'].apply(lambda x: zmiana_na_float(x))
    kolejnosc_kolumn=['Rodzaj nieruchomosci','Adres','Rynek','Powierzchnia (m2)','Powierzchnia działki (m2)',
                      'Cena (zł)','Cena (zł/m2)', 'Rodzaj zabudowy','Stan wykończenia', 'Opis' ]
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('Działka')].shape[0]>0:
    kolejnosc_kolumn=['Rodzaj nieruchomosci','Adres', 'Powierzchnia (m2)', 'Cena (zł)', 'Cena (zł/m2)', 
                      'Typ działki', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('użytkowy')].shape[0]>0:
    kolejnosc_kolumn=['Rodzaj nieruchomosci','Adres','Rynek','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)',
                      'Piętro','Stan wykończenia', 'Przeznaczenie lokalu', 'Rok budowy', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)


sciezka='/home/kinga/python/Web_scrapping/'+plik_wynikowy+'.xlsx'
# zapisanie do pliku Excel 
oferty.to_excel(sciezka, index=False)
print ('GOTOWE :) Plik został zapisany w lokalizacji: ', sciezka)


Wpisz adres strony internetowej: https://www.otodom.pl/sprzedaz/lokal/poznan/?search%5Bdist%5D=0&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1
Podaj, ile stron chcesz przejrzeć: 3
Podaj nazwę pliku, do którego chcesz zapisać: lokale_uzytkowe
Liczba wybranych stron to:  3
liczba dostępnych stron to:  15


GOTOWE :) Plik został zapisany w lokalizacji:  /home/kinga/python/Web_scrapping/lokale_uzytkowe.xlsx


In [11]:
oferty.head()

,Rodzaj nieruchomosci,Adres,Rynek,Powierzchnia (m2),Cena (zł),Cena (zł/m2),Piętro,Stan wykończenia,Przeznaczenie lokalu,Rok budowy,Opis
0,Lokal użytkowy na sprzedaż,"Poznań, Starołęka",pierwotny,180.00,1.6e+06,8888.89,parter,gotowy,NaN,2016,OpisNa sprzedaż lokal handlowo-usługowy o powi...
1,Lokal użytkowy na sprzedaż,"Poznań, Dębiec",pierwotny,257.27,1.3e+06,5053.06,parter,gotowy,NaN,2015,"OpisSprzedam lokal o powierzchni 257,27 m2 prz..."
2,Lokal użytkowy na sprzedaż,"Poznań, wielkopolskie",wtórny,845.70,3.25e+06,3842.97,NaN,NaN,NaN,1989,OpisOBIEKT PRZEMYSŁOWY - HALA MAGAZYNOWA - BIU...
3,Lokal użytkowy na sprzedaż,"Poznań, Stare Miasto",wtórny,68.00,600000,8823.53,parter,gotowy,NaN,2015,OpisSprzedam lokal użytkowy położony na parter...
4,Lokal użytkowy na sprzedaż,"Poznań, wielkopolskie",pierwotny,1100.00,5.5e+06,5000.00,parter,gotowy,NaN,2017,OpisOferuję do sprzedaży dochodową nieruchomoś...


## 6. Aktualizacja maj 2020

In [ ]:
# Import bibliotek
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep
import ipywidgets as widgets 
from datetime import date

# wykorzystana funkcja:

def zmiana_na_float (cecha):
    if isinstance (cecha, str): # jeśli cecha jest stringiem
        return float(cecha.rsplit(' ',1)[0].replace(' ','').replace(',','.'))     #rsplit-rozdzielam po spacji, ale tylko jednej spacji licząc od końca. Jeśli w stringu będzie kilka spacji, to podzieli tylko po tej ostatniej 
    else:
        return cecha


strona=input('Wpisz adres strony internetowej: ')
strony=[strona]
liczba_wybranych_str=int(input("Podaj, ile stron chcesz przejrzeć: "))
plik_wynikowy=input("Podaj nazwę pliku, do którego chcesz zapisać: ")

page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') # w tym jest klasa 'page-counter' i 'page_next'

przycisk_przewijania_stron=cala_strona.find_all(class_='pager-next')

for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_dostepnych_stron=item.find_all('li')
    liczba_dostepnych_stron=int(list(liczba_dostepnych_stron)[-2].get_text())
    
    print ('Liczba wybranych stron to: ', liczba_wybranych_str)
    print ('liczba dostępnych stron to: ', liczba_dostepnych_stron)

    # ograniczenie liczby pobranych stron do 10
    if liczba_wybranych_str<=liczba_dostepnych_stron and liczba_wybranych_str<=10:  
        for i in range(2,liczba_wybranych_str+1):
            strony.append(strona +'&page=' + str(i))
    else:
        for i in range(2,11):
            strony.append(strona +'&page=' + str(i))
        print('Maksymalnie możesz pobrać 10 stron- tyle zostanie pobranych!')
        

numery_ogloszen=[]
daty_dodania=[]
data_pobrania_oferty=date.today()
rodzaje_nieruchomosci=[]
adresy=[]
ceny=[]
dodatkowe=[]
opisy=[]
    
x=0
pasek_postepu = widgets.IntProgress(min=x, max=liczba_wybranych_str) # pasek postępu 
opis_paska_postepu= widgets.Label(value='Liczba pobranych stron',layout=widgets.Layout(width='auto') )
liczba_sparsowanych_stron =widgets.Label() 
kontener_paska_postepu= widgets.HBox([opis_paska_postepu,pasek_postepu, liczba_sparsowanych_stron])
display(kontener_paska_postepu)


for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert
    for item in wszystkie_oferty_items:
        try: 
            cena=zmiana_na_float(item.find(class_="offer-item-price").get_text().strip()) # strip - usuwam spacje na początku i końcu
        except:
            continue # niektóre ogłoszenia nie mają podanej ceny, pomijam te oferty
        
        rodzaj_nieruchomosci=item.find(class_='hidden-xs').get_text().split(':')[0]
        lokalizacja = item.find('span', {'class': 'hidden-xs'})
        powierzchnia= zmiana_na_float(item.find(class_="hidden-xs offer-item-area").get_text().split(' ')[0]) # żeby było bez m2
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') 
            try:
                zawartosc2=requests.get(adres_www)
            except:
                #print('brak url')
                break # przerywa i uruchamia kod od nowa, w przeciwnym wypadku pobrałby każdą ofertę 2 razy
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(id='root') 
            for item in szczegoly:       
                lokalizacja=item.find(class_='css-1ub7gzr').get_text().split(';}')[-1]
                try:
                    numer_oferty=item.find(class_='css-kos6vh').get_text().split(': ')[1]
                except:
                    'brak danych'
                try:
                    data_dodania=item.find(class_="css-lh1bxu").get_text().split(': ')[1].split('D')[0]
                except:
                    'brak danych'
                if soup2.find_all(class_='section-description'): 
                    opis=item.find(class_='section-description').get_text().split('underline;}')[1]
                else:
                    opis='brak informacji'   
                dodatkowe_parametry=soup2.find_all(class_='css-1ci0qpi')                               
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':', maxsplit=1)#maxsplit=1 dzieli tylko po pierwszym dwukropku
                        slownik[klucz]=wartosc

                    print('Rodzaj nieruchomości: ', rodzaj_nieruchomosci)
                    print('Lokalizacja: ', lokalizacja)
                    print('id: ',numer_oferty)
                    print('data_dodatnia: ', data_dodania)
                    print('Cena (zł): ', cena)
                    for klucz,wartosc in slownik.items():
                        print(klucz,': ',wartosc)
                    print('Opis: ', opis)
                    print(adres_www, '\n')
                    
                # zapisuję dane do takiej postaci, abym mogła je dalej zapisać w Pandas
                    numery_ogloszen.append(numer_oferty)
                    daty_dodania.append(data_dodania)
                    rodzaje_nieruchomosci.append(rodzaj_nieruchomosci)
                    adresy.append(lokalizacja)
                    ceny.append(cena)
                    opisy.append(opis)
                    dodatkowe.append(slownik)
    x=x+1
    pasek_postepu.value=x
    liczba_sparsowanych_stron.value=str(x) # Label musi być stringiem

oferty=pd.DataFrame()
for x in range(0,len(numery_ogloszen)):
    sr=pd.Series([numery_ogloszen[x], daty_dodania[x], data_pobrania_oferty,
                  rodzaje_nieruchomosci[x],adresy[x],ceny[x], opisy[x] ],\
                     index=['Nr oferty', 'Data dodania','Data pobrania oferty',
                            'Rodzaj nieruchomosci','Adres','Cena (zł)', 'Opis'])
    sr_dodatkowe=pd.Series(dodatkowe[x])
    sr=pd.concat([sr,sr_dodatkowe])
    oferty=pd.concat([oferty,sr],axis=1,ignore_index=True, sort=False)

oferty=oferty.T

# usuwam 'escape characters' z kolumny 'opis'
oferty.loc[:,'Opis'] = oferty.Opis.str.replace('\n','',regex=True)
# usuwam duplikaty
oferty.drop_duplicates(subset=['Nr oferty','Rynek','Powierzchnia', 'Cena (zł)' ], inplace=True)
oferty.rename(columns={'Powierzchnia': 'Powierzchnia (m2)'}, inplace=True)
oferty['Powierzchnia (m2)']=oferty['Powierzchnia (m2)'].apply(lambda x: zmiana_na_float(x)) # zmieniam powierzchnię na float i usuwam 'm2'
oferty['Cena (zł/m2)']=oferty.apply(lambda x: round(x['Cena (zł)']/x['Powierzchnia (m2)'],2),axis=1)

# Tworzę odpowiedni data frame w zależności od typu nieruchomości
if oferty[oferty['Rodzaj nieruchomosci'].str.contains('Mieszkanie')].shape[0]>0:
    kolejnosc_kolumn=['Nr oferty', 'Data dodania','Data pobrania oferty','Rodzaj nieruchomosci',
                      'Adres','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)','Liczba pokoi','Rynek', 
                      'Rodzaj zabudowy', 'Piętro','Liczba pięter', 'Materiał budynku','Okna', 'Ogrzewanie', 
                      'Rok budowy', 'Stan wykończenia', 'Czynsz', 'Forma własności','Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('Dom ')].shape[0]>0:
    oferty.rename(columns={'Powierzchnia działki': 'Powierzchnia działki (m2)'}, inplace=True)
    oferty['Powierzchnia działki (m2)']=oferty['Powierzchnia działki (m2)'].apply(lambda x: zmiana_na_float(x))
    kolejnosc_kolumn=['Nr oferty', 'Data dodania','Data pobrania oferty',
                      'Rodzaj nieruchomosci','Adres','Rynek','Powierzchnia (m2)','Powierzchnia działki (m2)',
                    'Cena (zł)','Cena (zł/m2)', 'Rodzaj zabudowy','Stan wykończenia', 'Opis' ]
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('Działka')].shape[0]>0:
    kolejnosc_kolumn=['Nr oferty', 'Data dodania','Data pobrania oferty',
                      'Rodzaj nieruchomosci','Adres', 'Powierzchnia (m2)', 'Cena (zł)', 'Cena (zł/m2)', 
                          'Typ działki', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)
elif oferty[oferty['Rodzaj nieruchomosci'].str.contains('użytkowy')].shape[0]>0:
    kolejnosc_kolumn=['Nr oferty', 'Data dodania','Data pobrania oferty',
                      'Rodzaj nieruchomosci','Adres','Rynek','Powierzchnia (m2)','Cena (zł)','Cena (zł/m2)',
                          'Piętro','Stan wykończenia', 'Przeznaczenie lokalu', 'Rok budowy', 'Opis']
    oferty=oferty.reindex(columns=kolejnosc_kolumn)

sciezka='/home/kinga/python/Web_scrapping/'+plik_wynikowy+'.xlsx'
# zapisanie do pliku Excel 
oferty.to_excel(sciezka, index=False)
print ('GOTOWE :) Plik został zapisany w lokalizacji: ', sciezka)

        